In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
pip install transformers

In [12]:
from transformers import TFBertModel,  BertConfig, BertTokenizerFast
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
import pandas as pd
from sklearn.model_selection import train_test_split

In [13]:
data = pd.read_csv('/content/drive/My Drive/MinorProject2/BERT2/hindi_tr.csv')
# data2 = pd.read_csv('/content/final.csv')

data = data[['text', 'task2']]
# data2 = data2[['text', 'task_2']]
# data2 = data2.rename(columns={'task_2': 'task2'})
# df_row = pd.concat([data, data2])

# data=df_row

data.head()

,text,task2
0,1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n...,HATE
1,"RT @Vishesh4: @jawaharyadavbjp जवाहर यादव, अगर...",NONE
2,RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही ...,HATE
3,Yey nina khothani labafazi benu phambili Finis...,PRFN
4,RT @Rajeshbhanjan2: जब भी कोई सिकुलर कोंग्रेसी...,HATE


In [14]:
data['task2'].value_counts()

NONE    2116
OFFN     465
HATE     234
PRFN     148
Name: task2, dtype: int64

In [15]:
data1=pd.read_csv("/content/drive/My Drive/MinorProject2/BERT2/hindi_train_2019.tsv",delimiter="\t")
data1.head()

,text_id,text,task_1,task_2,task_3
0,hasoc_hi_5556,"बांग्लादेश की शानदार वापसी, भारत को 314 रन पर ...",NOT,NONE,NONE
1,hasoc_hi_5648,सब रंडी नाच देखने मे व्यस्त जैसे ही कोई #शांती...,HOF,PRFN,UNT
2,hasoc_hi_164,तुम जैसे हरामियों के लिए बस जूतों की कमी है शु...,HOF,PRFN,TIN
3,hasoc_hi_3530,"बीजेपी MLA आकाश विजयवर्गीय जेल से रिहा, जमानत ...",NOT,NONE,NONE
4,hasoc_hi_5206,चमकी बुखार: विधानसभा परिसर में आरजेडी का प्रदर...,NOT,NONE,NONE


In [16]:
d=pd.read_csv("/content/drive/My Drive/MinorProject2/BERT2/hindi_test_1509.csv")
d.head()

,tweet_id,text,task1,task2,ID
0,1127781647783301120,@Kinjal_Dubey_ इसके लिए तैयारी हो रखी है\nआपको...,HOF,OFFN,hasoc_2020_hi_1938
1,1127492853171535872,RT @pratimamishra04: दिल्ली के मुख्यमंत्री @Ar...,NOT,NONE,hasoc_2020_hi_1201
2,1123638375896903681,"RT @nishantdabre2: आज एक तरफ खुशी है, तो एक तर...",NOT,NONE,hasoc_2020_hi_1628
3,1127496460310683648,@BJP4India @HardeepSPuri जिस कुनबे के लोग :-\n...,NOT,NONE,hasoc_2020_hi_3257
4,1127667277535399937,@manakgupta बादल....😂😂😂,NOT,NONE,hasoc_2020_hi_4133


In [17]:
data1['task_2'].value_counts()

NONE    2196
PRFN    1237
OFFN     676
HATE     556
Name: task_2, dtype: int64

In [18]:
data1=data1[['text','task_2']]
data1.head()

,text,task_2
0,"बांग्लादेश की शानदार वापसी, भारत को 314 रन पर ...",NONE
1,सब रंडी नाच देखने मे व्यस्त जैसे ही कोई #शांती...,PRFN
2,तुम जैसे हरामियों के लिए बस जूतों की कमी है शु...,PRFN
3,"बीजेपी MLA आकाश विजयवर्गीय जेल से रिहा, जमानत ...",NONE
4,चमकी बुखार: विधानसभा परिसर में आरजेडी का प्रदर...,NONE


In [19]:
data1=data1.rename(columns={"task_2": "task2"})
data1.head()

,text,task2
0,"बांग्लादेश की शानदार वापसी, भारत को 314 रन पर ...",NONE
1,सब रंडी नाच देखने मे व्यस्त जैसे ही कोई #शांती...,PRFN
2,तुम जैसे हरामियों के लिए बस जूतों की कमी है शु...,PRFN
3,"बीजेपी MLA आकाश विजयवर्गीय जेल से रिहा, जमानत ...",NONE
4,चमकी बुखार: विधानसभा परिसर में आरजेडी का प्रदर...,NONE


In [20]:
data3 = data1[data1['task2']=='PRFN']
data3.head()

,text,task2
1,सब रंडी नाच देखने मे व्यस्त जैसे ही कोई #शांती...,PRFN
2,तुम जैसे हरामियों के लिए बस जूतों की कमी है शु...,PRFN
20,इस मादरचोद को धुण्डके गांड मे गोली मरो @Uppolice,PRFN
22,You cry in front of ur god Of being deceived ...,PRFN
29,प्लीज कॉल मेरी जबरदस्ती चुदाई करो ना,PRFN


In [21]:
data4 = data1[data1['task2']=='HATE']
data4.head()

,text,task2
26,हद तो तब हो जाती हैं जब आरक्षण का लाभ लेने वा...,HATE
56,जल के बिना मछली जिंदा रह सकती है मगर बीजेपी टी...,HATE
92,जो भी मेरे ईश्वर के विरूद्ध आये काट दो सालों क...,HATE
100,तबरेज के लिए तो तेरे अमनपसंद कौम के अम्मि बेगम...,HATE
139,"हमको कोई फर्क नहीं पड़ता भाई,...कोई मंदिर तोड़...",HATE


In [22]:
data5=data1[data1['task2']=='OFFN']
data5.head()

,text,task2
18,"#नीच'समानार्थी शब्द #मोदी' दिला तर चालेल की, ल...",OFFN
58,एक शख्स ऐसा भी है जिसका नाम मोदी है पैर से ल...,OFFN
77,इस रशीदी जेहादी का पायजामा पीला तो नही है आज आ...,OFFN
120,"मुबारक हो, जितने भी हिजरे हैं वो दिल्ली में ही...",OFFN
121,एसे #नीच और #घटिया लोगो को #चौराहे पर सबके स...,OFFN


In [23]:
data=pd.concat([data,data3,data4,data5])
data.shape

(5432, 2)

In [24]:
data = data.sample(frac = 1)
data.head() 

,text,task2
3481,हरामी की ओलाद तेरे जिहादी मुलो ने मंदिर तोडा ह...,HATE
252,RT @Pinkimishr: एक बाबरी टूटी थी मुसलमान आज तक...,HATE
2080,RT @JITENDR78916715: @mshubhamtyagi @iamsolank...,NONE
2671,RT @Shubham62332811: #वेद_मेरा_भेद_है\n#love\n...,NONE
973,"@ZeeNewsHindi और कोई काम नहीं है इनके पास, बच्...",NONE


In [25]:
data['task2'] = pd.Categorical(data['task2'])
data['task2'] = data['task2'].cat.codes
data.head()

,text,task2
3481,हरामी की ओलाद तेरे जिहादी मुलो ने मंदिर तोडा ह...,0
252,RT @Pinkimishr: एक बाबरी टूटी थी मुसलमान आज तक...,0
2080,RT @JITENDR78916715: @mshubhamtyagi @iamsolank...,1
2671,RT @Shubham62332811: #वेद_मेरा_भेद_है\n#love\n...,1
973,"@ZeeNewsHindi और कोई काम नहीं है इनके पास, बच्...",1


In [26]:
data.shape

(5432, 2)

In [27]:
data, data_test = train_test_split(data, test_size = 0.30)
data.shape,data_test.shape

((3802, 2), (1630, 2))

In [28]:
model_name = 'bert-base-uncased'
max_length = 100
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [29]:
bert = transformer_model.layers[0]
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}
bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(bert_model, training=False)
task2 = Dense(units=len(data.task2.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='task2')(pooled_output)
outputs = {'task2': task2}
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')
model.summary()


Model: "BERT_MultiLabel_MultiClass"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 100)]             0         
_________________________________________________________________
bert (TFBertMainLayer)       ((None, 100, 768), (None, 109482240 
_________________________________________________________________
pooled_output (Dropout)      (None, 768)               0         
_________________________________________________________________
task2 (Dense)                (None, 4)                 3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


In [30]:
optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
loss = {'task2': CategoricalCrossentropy(from_logits = True)}
metric = {'task2': CategoricalAccuracy('accuracy')}

In [31]:
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)
y_issue = to_categorical(data['task2'])

In [32]:
x = tokenizer(
    text=data['text'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
history = model.fit(
    x={'input_ids': x['input_ids']},
    y={'task2': y_issue},
    validation_split=0.3,
    batch_size=64,
    epochs=30)

Epoch 1/30
42/42 [==============================] - 35s 836ms/step - loss: 1.1388 - accuracy: 0.5209 - val_loss: 1.1098 - val_accuracy: 0.5460
Epoch 2/30
42/42 [==============================] - 33s 778ms/step - loss: 1.0623 - accuracy: 0.5641 - val_loss: 1.0918 - val_accuracy: 0.5460
Epoch 3/30
42/42 [==============================] - 33s 777ms/step - loss: 1.0386 - accuracy: 0.5825 - val_loss: 1.0827 - val_accuracy: 0.5548
Epoch 4/30
42/42 [==============================] - 33s 776ms/step - loss: 1.0071 - accuracy: 0.6028 - val_loss: 1.0778 - val_accuracy: 0.5521
Epoch 5/30
42/42 [==============================] - 33s 777ms/step - loss: 0.9638 - accuracy: 0.6163 - val_loss: 1.0713 - val_accuracy: 0.5592
Epoch 6/30
42/42 [==============================] - 33s 776ms/step - loss: 0.9209 - accuracy: 0.6374 - val_loss: 1.0767 - val_accuracy: 0.5504
Epoch 7/30
42/42 [==============================] - 33s 775ms/step - loss: 0.8510 - accuracy: 0.6738 - val_loss: 1.0914 - val_accuracy: 0.5670

In [76]:
test_y_issue = to_categorical(data_test['task2'])
test_x = tokenizer(
    text=data_test['text'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)
model_eval = model.evaluate(
    x={'input_ids': test_x['input_ids']},
    y={'task2': test_y_issue}
)

51/51 [==============================] - 7s 135ms/step - loss: 1.7166 - accuracy: 0.5693


In [77]:
test_x

{'input_ids': <tf.Tensor: shape=(1630, 100), dtype=int32, numpy=
array([[  101, 19387,  1030, ...,     0,     0,     0],
       [  101,   100,  1329, ..., 29863, 29876,   102],
       [  101,  1332,  1315, ..., 29853,  1334,   102],
       ...,
       [  101,  1030, 12098, ...,     0,     0,     0],
       [  101,  1323, 29867, ...,     0,     0,     0],
       [  101, 19387,  1030, ...,     0,     0,     0]], dtype=int32)>}

In [78]:
preds = model.predict(test_x['input_ids'])

In [79]:
preds=preds['task2'].argmax(axis=-1)

In [80]:
preds

array([2, 0, 2, ..., 1, 0, 1])

In [81]:
test_y_issue=test_y_issue.argmax(axis=-1)

In [82]:
test_y_issue

array([2, 3, 2, ..., 2, 3, 1])

In [83]:
from sklearn.metrics import classification_report
print(classification_report(test_y_issue,preds))

              precision    recall  f1-score   support

           0       0.33      0.28      0.30       235
           1       0.70      0.74      0.72       646
           2       0.34      0.37      0.36       336
           3       0.68      0.63      0.65       413

    accuracy                           0.57      1630
   macro avg       0.51      0.51      0.51      1630
weighted avg       0.57      0.57      0.57      1630



In [96]:
d1=d[['text','task2']]
d1.head()

,text,task2
0,@Kinjal_Dubey_ इसके लिए तैयारी हो रखी है\nआपको...,OFFN
1,RT @pratimamishra04: दिल्ली के मुख्यमंत्री @Ar...,NONE
2,"RT @nishantdabre2: आज एक तरफ खुशी है, तो एक तर...",NONE
3,@BJP4India @HardeepSPuri जिस कुनबे के लोग :-\n...,NONE
4,@manakgupta बादल....😂😂😂,NONE


In [97]:
d1['task2'] = pd.Categorical(d1['task2'])
d1['task2'] = d1['task2'].cat.codes
d1.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,text,task2
0,@Kinjal_Dubey_ इसके लिए तैयारी हो रखी है\nआपको...,2
1,RT @pratimamishra04: दिल्ली के मुख्यमंत्री @Ar...,1
2,"RT @nishantdabre2: आज एक तरफ खुशी है, तो एक तर...",1
3,@BJP4India @HardeepSPuri जिस कुनबे के लोग :-\n...,1
4,@manakgupta बादल....😂😂😂,1


In [98]:
d1.shape

(663, 2)

In [99]:
test_y_issue1 = to_categorical(d1['task2'])
test_x1 = tokenizer(
    text=d1['text'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)
model_eval = model.evaluate(
    x={'input_ids': test_x1['input_ids']},
    y={'task2': test_y_issue1}
)

21/21 [==============================] - 3s 129ms/step - loss: 1.7208 - accuracy: 0.6169


In [100]:
label = model.predict(test_x1['input_ids'])

In [101]:
label=label['task2'].argmax(axis=-1)

In [102]:
print(label)

[2 2 1 2 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 2 1 1 1 1 1 1 1 2 1 1 2 1 1 2 1
 1 3 1 2 1 1 1 1 1 1 1 2 1 2 1 0 1 1 2 0 3 2 2 0 1 2 1 1 0 1 1 1 1 2 1 1 1
 1 2 1 1 0 1 1 1 1 2 0 2 1 1 1 1 1 2 2 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 2 1 1 3 2 1 1 1 1 1 2 1 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 3 1 2 1 1 1 1 2 1 2 2 1 1 1 1 2 2 2 2 1 1
 1 1 1 1 1 1 1 1 2 1 1 1 1 2 1 1 1 2 0 1 1 1 1 1 1 2 2 1 2 1 1 1 1 1 1 2 1
 1 1 2 1 2 1 1 0 1 1 1 1 2 1 1 1 1 1 1 1 1 1 0 1 2 1 2 1 1 1 1 1 1 0 1 1 2
 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 3 2 1 2 1 0 2 1 2 3 1 1 1 1 1 1 2 1 1
 1 1 1 1 1 1 3 2 3 1 1 2 1 1 1 1 3 1 1 0 1 2 1 1 1 1 1 0 1 1 1 1 1 1 3 1 1
 1 1 2 2 1 2 1 1 1 1 1 1 1 0 1 1 1 1 1 2 0 1 2 1 1 1 1 1 1 1 1 1 3 2 1 1 2
 1 1 1 2 1 1 1 1 1 1 1 1 2 2 1 1 1 1 2 1 1 1 1 3 1 1 2 1 1 2 3 2 1 1 1 1 1
 1 2 1 2 1 1 1 1 1 2 1 2 1 3 1 1 3 1 2 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 2 3 1
 1 1 1 1 1 2 1 2 1 1 2 1 2 1 1 1 1 1 1 0 2 1 0 1 1 2 1 2 1 1 1 1 2 1 2 0 1
 1 1 1 1 3 0 2 0 2 1 1 2 

In [103]:
d1['NewLabel'] = label 
d1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,text,task2,NewLabel
0,@Kinjal_Dubey_ इसके लिए तैयारी हो रखी है\nआपको...,2,2
1,RT @pratimamishra04: दिल्ली के मुख्यमंत्री @Ar...,1,2
2,"RT @nishantdabre2: आज एक तरफ खुशी है, तो एक तर...",1,1
3,@BJP4India @HardeepSPuri जिस कुनबे के लोग :-\n...,1,2
4,@manakgupta बादल....😂😂😂,1,1
...,...,...,...
658,RT @AishwaryVerma9: दिल्ली में शून्य थे और शून...,1,1
659,RT @KARUNASHANKEROJ: @sagarikaghose एक आर्टिकल...,1,1
660,@CM_P17 @PShatrujeet रमजान शुरू लेकिन मजाल है ...,2,1
661,RT @rukasa007: तो तुम्हारे सिर में एक गोली उता...,1,1


In [104]:
d1

,text,task2,NewLabel
0,@Kinjal_Dubey_ इसके लिए तैयारी हो रखी है\nआपको...,2,2
1,RT @pratimamishra04: दिल्ली के मुख्यमंत्री @Ar...,1,2
2,"RT @nishantdabre2: आज एक तरफ खुशी है, तो एक तर...",1,1
3,@BJP4India @HardeepSPuri जिस कुनबे के लोग :-\n...,1,2
4,@manakgupta बादल....😂😂😂,1,1
...,...,...,...
658,RT @AishwaryVerma9: दिल्ली में शून्य थे और शून...,1,1
659,RT @KARUNASHANKEROJ: @sagarikaghose एक आर्टिकल...,1,1
660,@CM_P17 @PShatrujeet रमजान शुरू लेकिन मजाल है ...,2,1
661,RT @rukasa007: तो तुम्हारे सिर में एक गोली उता...,1,1


In [105]:
NewLabel=d1['NewLabel'].map({0:"HATE",1:"NONE",2:"OFFN",3:"PRFN"})
NewLabel

0      OFFN
1      OFFN
2      NONE
3      OFFN
4      NONE
       ... 
658    NONE
659    NONE
660    NONE
661    NONE
662    NONE
Name: NewLabel, Length: 663, dtype: object

In [106]:
print(d1)

                                                  text  task2  NewLabel
0    @Kinjal_Dubey_ इसके लिए तैयारी हो रखी है\nआपको...      2         2
1    RT @pratimamishra04: दिल्ली के मुख्यमंत्री @Ar...      1         2
2    RT @nishantdabre2: आज एक तरफ खुशी है, तो एक तर...      1         1
3    @BJP4India @HardeepSPuri जिस कुनबे के लोग :-\n...      1         2
4                              @manakgupta बादल....😂😂😂      1         1
..                                                 ...    ...       ...
658  RT @AishwaryVerma9: दिल्ली में शून्य थे और शून...      1         1
659  RT @KARUNASHANKEROJ: @sagarikaghose एक आर्टिकल...      1         1
660  @CM_P17 @PShatrujeet रमजान शुरू लेकिन मजाल है ...      2         1
661  RT @rukasa007: तो तुम्हारे सिर में एक गोली उता...      1         1
662  @athavale_abhi @kavita_tewari गांधी परिवार और ...      0         1

[663 rows x 3 columns]
